In [2]:
# These define the maze. Walkable is boolean, if it is walkable or not
# Adjacent returns all adjacent nodes, together with the cost
# The 'input' variable on walkable is needed only for this day, it could change for your day
# For grid-mazes, you can probably copy one of the adjacent functions below
# For graph-mazes, you probably want walkable to always return True

def walkable(node, input=1362):
    # Returns true = open, false = wall
    x, y = node
    if x < 0 or y < 0:
        return False
    b = "{0:b}".format(x*x + 3*x + 2*x*y + y + y*y + input).count("1")
    return b % 2 == 0

# Diagonal allowed
def adjacent(node):
    # Returns list like [(adj_i, g_i)] where g_i is cost of going from node to adj_i
    x, y = node
    return [((x+a, y+b), 1) for a in range(-1, 2) for b in range(-1, 2) if a != 0 or b != 0]

# Horizontal only
def adjacent(node):
    # Returns list like [(adj_i, g_i)] where g_i is cost of going from node to adj_i
    x, y = node
    return [((x+a, y+b), 1) for a in range(-1, 2) for b in range(-1, 2) if (a != 0) != (b != 0)]

In [18]:
# Draw a grid-maze for non-negative coordinates
# from (0, 0) to (size-1, size-1)
# Draw '.' for walkable, '#' for non-walkable
import math
size = 10
size_size = math.ceil(math.log10(size))

print(" " + " " * size_size, end="")
[print(str(y)[-1], end="") for y in range(size)]
for x in range(size):
    print(f"\n{str(x).zfill(size_size)} ", end="")
    for y in range(size):
        print("." if walkable((y, x), input) else "#", end="")

  0123456789
0 #..###.#..
1 #.#..#.##.
2 #..#.#.##.
3 ##..##.##.
4 ..#.##..##
5 #.........
6 ###.##.###
7 #.#..#.##.
8 .###......
9 .####..###

In [62]:
# A* overview: we keep two lists of 'nodes' ((x, y) tuples)
#   open nodes and closed nodes.
# Also, for each node we keep a current cost to get there, and we can calculate a
#   heuristic to get to the target (e.g. manhattan).
#   We call this F = G (get there) + H (heuristic)
# Add the starting node to the list of open nodes

# Then repeat:
# - pick the open node with lowest F and switch it to the closed list. it's the 'current'
# - for each node adjacent to the current node:
#   - If it's a wall or in the closed list: ignore/skip
#   - If it's not on the open list, add it to the open list. calculate F for it
#   - If it is on the open list, check to see if F is lower from here, if so, update F

# Stop when:
# - You add the target node to the closed list, in which case the path is found.
# - The open list is empty, there is no path to the target node


In [3]:
# To adapt this code to a different excercise:
# - Implement 'walkable' and 'adjacent' functions
# - Define start and target nodes below
# - Implement heuristic function
# - (double check that if x is a node, 'x in list' works (it does for tuples of ints))
start = (1, 1)
target = (31, 39)

# heuristic. IMPORTANT! must be NOT be an overestimation of the true distance
# on a grid, if you can use diagonals, use Euclidean
# on a grid, if you cannot use diagonals, use Manhattan
# Interestingly, if h = 0, we get Dijkstra's algorithm
h = lambda node: abs(node[0] - target[0]) + abs(node[1] - target[1])

open = [start]
closed = []
open_parents, closed_parents = [None], []
f = [(0, h(start))] # (g, h)

while len(open) > 0:
    current_ix = f.index(min(f, key=sum))
    current_node, current_f = open[current_ix], f.pop(current_ix)
    closed.append(open.pop(current_ix))
    closed_parents.append(open_parents.pop(current_ix))

    print(f"Now considering {current_node}. Length so far: {current_f[0]}")

    if current_node == target:
        print(f"Final path length: {sum(current_f)}")
        break

    # Here are our two day-specific functions, edit their signature if neccecary
    for adj, adj_g in adjacent(current_node):
        if not walkable(adj) or adj in closed:
            continue
        if adj not in open:
            new_f = (current_f[0] + adj_g, h(adj))
            open.append(adj)
            f.append(new_f)
            open_parents.append(current_node)
        else:
            ix = open.index(adj)
            new_f = (current_f[0] + adj_g, h(adj))
            if sum(new_f) < sum(f[ix]):
                open_parents[ix] = current_node
                f[ix] = new_f


Now considering (1, 1). Length so far: 0
Now considering (1, 2). Length so far: 1
Now considering (2, 2). Length so far: 2
Now considering (2, 3). Length so far: 3
Now considering (3, 3). Length so far: 4
Now considering (3, 4). Length so far: 5
Now considering (3, 5). Length so far: 6
Now considering (3, 6). Length so far: 7
Now considering (4, 5). Length so far: 7
Now considering (3, 7). Length so far: 8
Now considering (5, 5). Length so far: 8
Now considering (4, 7). Length so far: 9
Now considering (6, 5). Length so far: 9
Now considering (4, 8). Length so far: 10
Now considering (6, 6). Length so far: 10
Now considering (7, 5). Length so far: 10
Now considering (5, 8). Length so far: 11
Now considering (6, 7). Length so far: 11
Now considering (8, 5). Length so far: 11
Now considering (5, 9). Length so far: 12
Now considering (6, 8). Length so far: 12
Now considering (9, 5). Length so far: 12
Now considering (6, 9). Length so far: 13
Now considering (7, 8). Length so far: 13
Now c

In [20]:
# Backtrack the path (returns a list of nodes, from end to beginning)
# Length of this list will be path length + 1 (since it's inclusive)
def backtrack(end, nodes, parents):
    node, path = end, [end]
    while node is not None:
        node = parents[nodes.index(node)]
        path.append(node)
    return path[0:-1]

len(backtrack((31, 39), closed, closed_parents))

83

In [82]:
start = (1, 1)

# This is part 2, which is trying to calculate 'how many locations can be reached'
# so the heuristic is 0 (since we don't have a target), and we want to get all nodes with g <= 50
h = lambda node: 0
open = [start]
closed = []
f = [(0, h(start))] # (g, h)

while len(open) > 0:

    if len(f) == 0:
        break

    current_ix = f.index(min(f, key=sum))
    current_node, current_f = open[current_ix], f.pop(current_ix)

    if current_f[0] > 50:
        # This node cannot be reached in 50 steps under any circumstances
        # so from now on, it is irrelevant for us, dont add it to the closed list
        continue

    closed.append(open.pop(current_ix))

    for adj, adj_g in adjacent(current_node):
        if not walkable(adj) or adj in closed:
            continue
        if adj not in open: # 'in' may not work if adj is a more complex data type?
            new_f = (current_f[0] + adj_g, h(adj))
            open.append(adj)
            f.append(new_f)
        else:
            ix = open.index(adj)
            new_f = (current_f[0] + adj_g, h(adj))
            if sum(new_f) < sum(f[ix]):
                f[ix] = new_f

print(len(closed))

138
